In [1]:
DATA_NAME = 'syntheticMRI3D-full' 
TRANSFORM = 'wavelet'
CHANNEL = 'dad'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_dad_wavelet_full_syntheticMRI3D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,7067273.80,-219.54,13.23,-2.83,-2.59,0.33
4,0.0,-219.54,2772116.76,-12.73,7.29,2.44,0.37
5,0.0,13.23,-12.73,342527.43,-0.14,0.10,0.17
6,0.0,-2.83,7.29,-0.14,51637.50,0.06,0.08
7,0.0,-2.59,2.44,0.10,0.06,2876.78,-0.03
8,0.0,0.33,0.37,0.17,0.08,-0.03,55.84


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_22144/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,-0.00005,0.00001,-0.00000,-0.00002,0.00002
4,NaN,-0.00005,1.00000,-0.00001,0.00002,0.00003,0.00003
5,NaN,0.00001,-0.00001,1.00000,-0.00000,0.00000,0.00004
6,NaN,-0.00000,0.00002,-0.00000,1.00000,0.00000,0.00004
7,NaN,-0.00002,0.00003,0.00000,0.00000,1.00000,-0.00007
8,NaN,0.00002,0.00003,0.00004,0.00004,-0.00007,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

311.80385305952996

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.17158289e+12 1.06267012e+12 1.56550436e+10 3.32928417e+08
 9.19313483e+05 2.59842691e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.0,0.997031,-0.076592,-0.007787,-0.001172,-0.000066,-0.000001
1,0.0,0.076364,0.996740,-0.025755,-0.003809,-0.000211,-0.000004
2,0.0,0.009671,0.024922,0.998895,-0.038602,-0.002089,-0.000040
3,-0.0,0.001832,0.004667,0.038436,0.999073,-0.018713,-0.000360
4,0.0,0.000136,0.000345,0.002800,0.018613,0.999775,-0.009726
5,-0.0,0.000004,0.000010,0.000081,0.000539,0.009717,0.999953
6,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.9236356854028189,
 0.9750783163818559,
 0.9615641290779193,
 0.9813869251614514,
 0.9902825224639251,
 1.0]